### Step 1. Importing the required libraries

In [31]:
from flask import Flask, render_template, session, url_for, redirect, request
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import NumberRange
import numpy as np
from tensorflow.keras.models import load_model
import joblib

### Step 2. Defining a function for returning our model’s prediction


In [32]:
def return_prediction(model,scaler,sample_json):
 
    s_len = sample_json['sepal_length']
    s_wid = sample_json['sepal_width']
    p_len = sample_json['petal_length']
    p_wid = sample_json['petal_width']

    flower = [[s_len,s_wid,p_len,p_wid]]
    flower = scaler.transform(flower)

    classes = np.array(['setosa', 'versicolor', 'virginica'])
    class_ind = model.predict_classes(flower)

    return classes[class_ind][0]

### Step 3. Defining the app routes and completing the app.py file

In [40]:
app9 = Flask(__name__)
# Configure a secret SECRET_KEY
app9.config['SECRET_KEY'] = 'someRandomKey'

# Loading the model and scaler
flower_model = load_model("final_iris_model.h5")
flower_scaler = joblib.load("iris_scaler.pkl")

# Now create a WTForm Class
class FlowerForm(FlaskForm):
    sep_len = StringField('Sepal-Length : ')
    sep_wid = StringField('Sepal-Width : ')
    pet_len = StringField('Petal-Length : ')
    pet_wid = StringField('Petal-Width : ')
    submit = SubmitField('Analyze Data')
    
@app9.route('/', methods=['GET', 'POST'])
def index():
    # Create instance of the form.
    form = FlowerForm()
    
    # If the form is valid on submission
    if form.validate_on_submit():
        # Grab the data from the input on the form.
        session['sep_len'] = form.sep_len.data
        session['sep_wid'] = form.sep_wid.data
        session['pet_len'] = form.pet_len.data
        session['pet_wid'] = form.pet_wid.data
        
        return redirect(url_for("prediction"))
    
    return render_template('home.html', form=form)

@app9.route('/prediction')
def prediction():
    #Defining content dictionary
    content = {}
    content['sepal_length'] = float(session['sep_len'])
    content['sepal_width'] = float(session['sep_wid'])
    content['petal_length'] = float(session['pet_len'])
    content['petal_width'] = float(session['pet_wid'])

    result = return_prediction(model=flower_model,scaler=flower_scaler,sample_json=content)
    
    return render_template('prediction.html',result=result)

app9.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2021 23:12:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2021 23:12:50] "POST / HTTP/1.1" 302 -
C:\Users\2016m\Anaconda3\envs\TensorFlow_kp\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [01/Dec/2021 23:12:50] "GET /prediction HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2021 23:13:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2021 23:13:38] "POST / HTTP/1.1" 302 -
C:\Users\2016m\Anaconda3\envs\TensorFlow_kp\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [01/Dec/2021 23:13:38] "GET /prediction HTTP/1.1" 200 -
